## This notebook merge external attributes for 2023 - 2027

In [1]:
import pandas as pd
import os
import geopandas as gpd
import glob
import os

# merge 2013 - 2022 dataset
path = r'../../data/curated/merged_dataset/' # use your path
all_files = glob.glob(os.path.join(path , "*.csv"))

li = []

for filename in sorted(all_files):
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

data = pd.concat(li, axis=0, ignore_index=True)


In [2]:
# create path
new_path = '../../data/curated/2023_2027_data/'

if not os.path.exists(new_path):
    os.makedirs(new_path)

In [3]:
''' since 2022 dataset has the most values of postcode, sa2 2021 and sa2 2016, and includes all values from previous years of those attributes, postcode, sa2 2021 and sa2 2016 
from 2022 dataset will be used for further prediction'''
#data = pd.read_csv(f'../../data/curated/merged_dataset/2022_merged_data.csv')
post_sa2 = data[['postcode', 'sa2_2021', 'sa2_2016']]
post_sa2 = post_sa2.drop_duplicates().reset_index().drop(columns=['index'])
post_sa2

,postcode,sa2_2021,sa2_2016
0,3723,204011057,204011057
1,3859,205051101,205051101
2,3555,202011022,202011022
3,3145,208041195,208041195
4,3166,212051326,212051326
...,...,...,...
1601,3477,201031013,201031013
1602,3355,213051583,213051365
1603,3127,207031165,207031165
1604,3068,207031163,207031163


In [4]:
# load predicted external attributes
pred_gdp_sr = pd.read_csv('../../data/curated/feature_prediction/21_27_gdp_with_saving_predicted.csv')
pred_income = pd.read_csv('../../data/curated/feature_prediction/20_27_income_per_person_2016sa2.csv', index_col=[0])
pred_population = pd.read_csv('../../data/curated/feature_prediction/22_27_population.csv', index_col=[0])
pred_population = pred_population.rename(columns={'SA2 code': 'sa2_2021'})
pred_crime_cases = pd.read_csv('../../data/curated/feature_prediction/23_27_crime_case.csv', index_col=[0])
pred_crime_cases = pred_crime_cases.rename(columns={'Postcode': 'postcode'})

In [5]:
year = [2023, 2024, 2025, 2026, 2027]

for i in year:

    # store year, sa2 (2021 and 2016), postcode
    data = post_sa2
    data['year'] = i

    # store gdp and saving rate into the dataframe
    gdp = list(pred_gdp_sr.loc[pred_gdp_sr['year'] == i, 'gdp'])[0]
    sr = list(pred_gdp_sr.loc[pred_gdp_sr['year'] == i, 'saving'])[0]
        
    data['gdp(USD Millioins)'] = gdp
    data['saving_rate(% of GDP)'] = sr

    # store income per person per year into the dataframe
    income_data = pred_income.loc[pred_income['Year'] == i][['sa2_2016', 'income_per_person_sa2']]
    data['income_per_person'] = (data.merge(income_data, on=['sa2_2016'], how='left')['income_per_person_sa2'])

    # store population density into the dataframe
    population_data = pred_population.loc[pred_population['year'] == i][['sa2_2021', 'pred']]
    data['population_density'] = (data.merge(population_data, on=['sa2_2021'], how='left')['pred'])

    # store crime cases into the dataframe
    crime_cases_data = pred_crime_cases.loc[pred_crime_cases['Year'] == i][['postcode', 'crime_count']]
    data['crime_cases'] = (data.merge(crime_cases_data, on=['postcode'], how='left')['crime_count'])

    # output csv file of merged external data
    data.to_csv(f'../../data/curated/2023_2027_data/{i}_data.csv')

In [6]:
import numpy as np
# assume in future years, the properties will not move and have the same proximity to all places, the most recent 2022 was taken
distance_df = pd.read_csv(f'../../data/curated/merged_dataset/2022_merged_data.csv')[
    ['year', 'sa2_2021', 'residence_type', 'nbed', 'nbath', 'ncar', 'min_distance_to_cbd', 
    'min_distance_to_park', 'min_distance_to_prim',
    'min_distance_to_second', 'min_distance_to_train',
    'min_distance_to_hosp', 'min_distance_to_poli', 'min_distance_to_shop']]

distance_df = distance_df.groupby(['sa2_2021', 'residence_type', 'nbed', 'nbath', 'ncar'], as_index=False).agg({
'min_distance_to_cbd': 'mean', 'min_distance_to_park': 'mean', 'min_distance_to_prim': 'mean', 
'min_distance_to_second': 'mean', 'min_distance_to_train': 'mean', 'min_distance_to_hosp': 'mean', 
'min_distance_to_poli': 'mean', 'min_distance_to_shop': 'mean'})

beds = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]
baths = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]
cars = [0, 1, 2, 3, 4, 5, 6, 7, 8]
sa2 = list(distance_df['sa2_2021'].unique())
sa2_len = len(sa2)
df1,df2,df3 = [pd.DataFrame({name:lst} ) for lst,name in zip([beds,baths,cars], ['nbed', 'nbath', 'ncar'])]

df_123 = df1.join(df2, how='cross').join(df3, how='cross')

internal_df = df_123.query("nbed >= nbath >= ncar").reset_index(drop=True)
sa2 = np.repeat(sa2, len(internal_df))
internal_df = pd.concat([internal_df]*sa2_len, ignore_index=True)

# assign internal values to sa2 codes
internal_df['sa2_2021'] = sa2
internal_df

for i in year:
    external_df = pd.read_csv(f'../../data/curated/2023_2027_data/{i}_data.csv').iloc[:, 1:]
    # sum up crime cases for each sa2 district
    external_df = external_df.groupby(['year', 'sa2_2021', 'sa2_2016', 'gdp(USD Millioins)', 
    'saving_rate(% of GDP)', 'income_per_person', 'population_density'], as_index=False).agg({'crime_cases': 'sum'})
    partial_df = external_df.merge(distance_df, on='sa2_2021', how='inner').drop(['sa2_2016'], axis=1)
    #full_predict = internal_df.merge(partial_df, on='sa2_2021', how='inner')
    #full_predict.to_csv(f'../../data/curated/2023_2027_data/{i}_data.csv', index=False)
    partial_df.to_csv(f'../../data/curated/2023_2027_data/{i}_data.csv', index=False)

In [9]:
distance_df = pd.read_csv(f'../../data/curated/merged_dataset/2022_merged_data.csv')[
    ['year', 'sa2_2021', 'residence_type', 'nbed', 'nbath', 'ncar', 'min_distance_to_cbd', 
    'min_distance_to_park', 'min_distance_to_prim',
    'min_distance_to_second', 'min_distance_to_train',
    'min_distance_to_hosp', 'min_distance_to_poli', 'min_distance_to_shop']]

distance_df = distance_df.groupby(['sa2_2021', 'residence_type', 'nbed', 'nbath', 'ncar'], as_index=False).agg({
'min_distance_to_cbd': 'mean', 'min_distance_to_park': 'mean', 'min_distance_to_prim': 'mean', 
'min_distance_to_second': 'mean', 'min_distance_to_train': 'mean', 'min_distance_to_hosp': 'mean', 
'min_distance_to_poli': 'mean', 'min_distance_to_shop': 'mean'})

distance_df

,sa2_2021,residence_type,nbed,nbath,ncar,min_distance_to_cbd,min_distance_to_park,min_distance_to_prim,min_distance_to_second,min_distance_to_train,min_distance_to_hosp,min_distance_to_poli,min_distance_to_shop
0,201011001,Apartment,1.0,1.0,1,119.437573,1.704810,0.977733,114.770160,90.085910,140.56888,74.356080,13.6492
1,201011001,Apartment,2.0,1.0,1,119.732770,1.762795,1.159870,114.770160,90.085910,140.56888,74.356080,13.6492
2,201011001,Apartment,2.0,1.0,2,119.406390,2.709500,1.982420,114.770160,90.085910,140.56888,74.356080,13.6492
3,201011001,House,2.0,2.0,1,120.125710,0.664800,0.093980,114.770160,90.085910,140.56888,74.356080,13.6492
4,201011001,House,3.0,1.0,1,120.898328,2.077348,0.828286,114.770160,90.085910,140.56888,74.356080,13.6492
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10363,217041480,House,4.0,1.0,2,259.682420,0.361740,2.075460,0.323100,1.374450,0.77911,1.046480,13.6492
10364,217041480,House,4.0,2.0,1,257.989052,1.164082,0.988412,2.799602,1.877564,2.13502,1.820720,13.6492
10365,217041480,House,4.0,2.0,2,257.317120,1.176300,1.632986,3.795584,3.881284,4.34941,3.407803,13.6492
10366,217041480,House,4.0,2.0,3,255.248780,0.955490,0.361790,4.678840,4.555660,5.40752,4.017570,13.6492


In [7]:
external_df = pd.read_csv(f'../../data/curated/2023_2027_data/2023_data.csv').iloc[:, 1:]
# sum up crime cases for each sa2 district
external_df = external_df.groupby(['year', 'sa2_2021', 'sa2_2016', 'gdp(USD Millioins)', 
'saving_rate(% of GDP)', 'income_per_person', 'population_density'], as_index=False).agg({'crime_cases': 'sum'})
external_df

,year,sa2_2021,sa2_2016,gdp(USD Millioins),saving_rate(% of GDP),income_per_person,population_density,crime_cases
0,2023,201011001,201011001,3567759,13.581,68806.001600,630.0,7754
1,2023,201011002,201011002,3567759,13.581,80387.628305,1377.0,8920
2,2023,201011005,201011005,3567759,13.581,69507.858889,568.0,8537
3,2023,201011006,201011006,3567759,13.581,60341.452446,586.0,9793
4,2023,201011007,201011007,3567759,13.581,62852.528150,449.0,1146
...,...,...,...,...,...,...,...,...
504,2023,217031476,217031476,3567759,13.581,53241.174674,404.0,1234
505,2023,217041477,217041477,3567759,13.581,58321.216779,421.0,5342
506,2023,217041478,217041478,3567759,13.581,60052.461643,433.0,1110
507,2023,217041479,217041479,3567759,13.581,62351.625823,723.0,5852


In [9]:
partial_df = distance_df.merge(external_df, on='sa2_2021', how='inner').drop(['sa2_2016'], axis=1)
partial_df

,sa2_2021,residence_type,nbed,nbath,ncar,min_distance_to_cbd,min_distance_to_park,min_distance_to_prim,min_distance_to_second,min_distance_to_train,min_distance_to_hosp,min_distance_to_poli,min_distance_to_shop,year,gdp(USD Millioins),saving_rate(% of GDP),income_per_person,population_density,crime_cases
0,201011001,Apartment,1.0,1.0,1,119.437573,1.704810,0.977733,114.770160,90.085910,140.56888,74.356080,13.6492,2023,3567759,13.581,68806.001600,630.0,7754
1,201011001,Apartment,2.0,1.0,1,119.732770,1.762795,1.159870,114.770160,90.085910,140.56888,74.356080,13.6492,2023,3567759,13.581,68806.001600,630.0,7754
2,201011001,Apartment,2.0,1.0,2,119.406390,2.709500,1.982420,114.770160,90.085910,140.56888,74.356080,13.6492,2023,3567759,13.581,68806.001600,630.0,7754
3,201011001,House,2.0,2.0,1,120.125710,0.664800,0.093980,114.770160,90.085910,140.56888,74.356080,13.6492,2023,3567759,13.581,68806.001600,630.0,7754
4,201011001,House,3.0,1.0,1,120.898328,2.077348,0.828286,114.770160,90.085910,140.56888,74.356080,13.6492,2023,3567759,13.581,68806.001600,630.0,7754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10409,217041480,House,4.0,1.0,2,259.682420,0.361740,2.075460,0.323100,1.374450,0.77911,1.046480,13.6492,2023,3567759,13.581,63433.996482,519.0,3795
10410,217041480,House,4.0,2.0,1,257.989052,1.164082,0.988412,2.799602,1.877564,2.13502,1.820720,13.6492,2023,3567759,13.581,63433.996482,519.0,3795
10411,217041480,House,4.0,2.0,2,257.317120,1.176300,1.632986,3.795584,3.881284,4.34941,3.407803,13.6492,2023,3567759,13.581,63433.996482,519.0,3795
10412,217041480,House,4.0,2.0,3,255.248780,0.955490,0.361790,4.678840,4.555660,5.40752,4.017570,13.6492,2023,3567759,13.581,63433.996482,519.0,3795
